# Tool Calling

This notebook demonstrate tool calling on chat/completions endpoint using openai interface.


In [6]:
from openai import OpenAI
import getpass

api_key = getpass.getpass()

# Create a new api key with your account if the one below is not working
client = OpenAI(
    api_key=api_key,
    base_url="https://labs.liquid.ai/api/v1",
)

Load the tools


In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. <city>, <state>",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. <city>, <state>",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    },
                },
                "required": ["location", "format", "num_days"],
            },
        },
    },
]

### Trigger tool call in one message


Tool calling returns no content, but a tool_calls object is returned in the response. This object contains information about the function name and arugments that will be called later.


In [8]:
messages = []
messages.append(
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    }
)
# This message will trigger get_n_day_weather_forecast
messages.append(
    {
        "role": "user",
        "content": "what is the weather going to be like in Glasgow, Scotland over the next 5 days",
    }
)

response = client.chat.completions.create(
    model="liquid-preview-0.1",
    messages=messages,
    tools=tools,
)

print(response)

ChatCompletion(id='cmpl-ce80c1a3a35d4aefa6ff1164d080ea6e', choices=[Choice(finish_reason='tool_calls', index=None, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_79721a8fc37044ff95f74377f9f2608f', function=Function(arguments={'location': 'Glasgow, Scotland', 'format': 'celsius', 'num_days': 5}, name='get_n_day_weather_forecast'), type='function')]))], created=66140, model='liquid-preview-0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=74, prompt_tokens=630, total_tokens=704), systen_fingerprint=None)


### Trigger another tool call

Depending on the prompt, the model may choose to use a different function to better fit the context.


In [9]:
messages = []
# This message will trigger get_current_weather instead.
messages.append(
    {
        "role": "user",
        "content": "what is the current weather in milpitas?",
    }
)

response = client.chat.completions.create(
    model="liquid-preview-0.1",
    messages=messages,
    tools=tools,
)
print(response)

ChatCompletion(id='cmpl-d780da3d97f844fb89640978957ee605', choices=[Choice(finish_reason='tool_calls', index=None, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_130107736e7848d08b9c51113d3b9e60', function=Function(arguments={'location': 'Milpitas', 'format': 'celsius'}, name='get_current_weather'), type='function')]))], created=66279, model='liquid-preview-0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=43, prompt_tokens=591, total_tokens=634), systen_fingerprint=None)


### Agent will ask for clarification if the user request is ambiguous

In this example, the tool is not called. The agent asks for clarification on the user's request.


In [11]:
messages = []
messages.append(
    {
        "role": "user",
        "content": "what is the weather going to be like in the next 5 days",
    }
)


def message_assistant(messages):
    response = client.chat.completions.create(
        model="liquid-preview-0.1",
        messages=messages,
        tools=tools,
    )
    return response


response = message_assistant(messages)
print(response)

ChatCompletion(id='cmpl-ff10920d2a224d6984b254a4c5e26ded', choices=[Choice(finish_reason='stop', index=None, logprobs=None, message=ChatCompletionMessage(content='To provide the weather forecast for the next 5 days, I will need to know the location for which you want the forecast. Could you please specify a city and state?', role='assistant', function_call=None, tool_calls=None))], created=66338, model='liquid-preview-0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=595, total_tokens=631), systen_fingerprint=None)


The Agent returns a tool call when gathered enough information for the tool.


In [12]:
messages.append(
    {
        "role": "user",
        "content": "Milpitas CA",
    }
)
response = message_assistant(messages)
print(response)

ChatCompletion(id='cmpl-fd6e1a286d6c4b8b91be5df2589cd1bd', choices=[Choice(finish_reason='tool_calls', index=None, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1af4c1ebc1c942f58d5547d7efc9f4ce', function=Function(arguments={'location': 'Milpitas, CA', 'format': 'celsius', 'num_days': 5}, name='get_n_day_weather_forecast'), type='function')]))], created=66378, model='liquid-preview-0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=59, prompt_tokens=605, total_tokens=664), systen_fingerprint=None)
